In [ ]:
from mx.progress import init_with_progress
init_with_progress()
from mx.prelude import *
# tf.config.run_functions_eagerly(True)
# tf.data.experimental.enable_debug_mode()

In [ ]:
from mx import train

rand_run_name = u.random_run_name()
run_name_options = [
    Box(
        default=True,
        name="dev",
        desc="Interactive development",
    ),
    Box(
        name="random",
        desc=f"'{rand_run_name}' (Random name)",
    ),
    Box(
        name="prev",
        desc="Existing previous run",
    ),
    Box(
        name="interactive",
        desc="Interactive",
    ),
]
run_name = choose_from_iterator(run_name_options, "Run Name")
if run_name.name == "dev":
    u.set_run_name("dev")
    force_new = True
    force_not_new = False
elif run_name.name == "prev":
    u.set_run_name("blessed")
    force_new = False
    force_not_new = True
elif run_name.name == "random":
    u.set_run_name(rand_run_name)
    force_new = True
    force_not_new = False
else:
    # default run name is "interactive-<date>"
    force_new = False
    force_not_new = False

pipeline = choose_pipeline()

output_dir = pipeline.output_dir()

if pipeline.use_float16:
    tf.keras.mixed_precision.set_global_policy("mixed_float16")

if isinstance(pipeline.dataset, mxd.MxMNIST):
    u.setregtype('mnist')
elif isinstance(pipeline.dataset, mxd.BvhDataset):
    u.setregtype('bvh')

model = pipeline.make_or_load_model(force_new=force_new, force_not_new=force_not_new)
loss_fn = pipeline.make_loss_fn()
train_data, val_data = pipeline.make_train_data()
predict_fn = pipeline.task.make_predict_fn(model)
vizs = pipeline.dataset.get_visualizations(model, output_dir)
# vizr = Visualizer(
#     visualizations=vizs,
#     configs={},
# )
vizr = None
train_loop = train.make_train_loop(
    model=model,
    loss_fn=loss_fn,
    data=train_data,
    val_data=val_data,
    output_dir=output_dir,
    vizr=vizr,
    log_interval="never",
)

In [ ]:
from mx.progress import create_progress_manager


if force_new:
    train_loop()
